In [1]:
import pandas as pd

In [2]:
# import bikes_df and yelp_df
bikes_df =pd.read_csv(r"B:\LHL\StatsModeling_Project\LHL-statsModeling-priject\data\bikes_df.csv",sep=',')
places_df =pd.read_csv(r"B:\LHL\StatsModeling_Project\LHL-statsModeling-priject\data\places_df.csv",sep=',')

bikes_df = bikes_df.rename(columns={"id":"station_id", 'name':'station_name', 'latitude': 'station_latitude', 'longitude':'station_longitude'})
places_df = places_df.rename(columns={'name':'place_name','id':'place_id','rating':'place_rating','latitude': 'place_latitude', 'longitude':'place_longitude'})

Join the data from Part 1 with the data from Part 2 to create a new dataframe.

In [13]:
len(places_df)

1185

In [20]:
df = places_df.join(bikes_df.set_index('station_id'), how='left',on='station_id')
df = df[['station_name','place_name','bar','dog_park','restaurant','distance', 'place_latitude','place_longitude', 'station_latitude', 'station_longitude', 'slots', 'free_bikes','empty_slots','place_rating']]
df = df.reset_index(drop=True)
df = df.dropna()
# save dataframe as CSV to access in other notebooks
df.to_csv(r'B:\LHL\StatsModeling_Project\LHL-statsModeling-priject\data\joinned_df.csv',sep =',',index=False)
df

,station_name,place_name,bar,dog_park,restaurant,distance,place_latitude,place_longitude,station_latitude,station_longitude,slots,free_bikes,empty_slots,place_rating
1070,Riverside Museum,Dukes,1,0,0,838.518390,55.866966,-4.292194,55.865683,-4.305367,6,6,0,4.4
1071,Riverside Museum,BrewDog Glasgow,1,0,0,871.079804,55.867702,-4.291878,55.865683,-4.305367,6,6,0,4.3
1072,Riverside Museum,6 Degrees North,1,0,0,854.560573,55.870835,-4.315485,55.865683,-4.305367,6,6,0,4.2
1073,Riverside Museum,Nam Tuk,0,0,1,596.695005,55.869846,-4.299887,55.865683,-4.305367,6,6,0,4.7
1074,Parkhead,Old Black Bull,1,0,0,636.954617,55.852070,-4.200545,55.846524,-4.197475,8,4,4,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1180,Knightswood Park,Lincoln Inn,1,0,0,553.814985,55.899938,-4.349245,55.895974,-4.354877,10,1,9,2.0
1181,Knightswood Park,New Dragon Inn,1,0,0,818.569003,55.896633,-4.341972,55.895974,-4.354877,10,1,9,0.0
1182,Fullarton,Kimberley,1,0,0,678.130598,55.843273,-4.170313,55.837175,-4.170209,12,6,6,0.0
1183,Fullarton,Elders Bar,1,0,0,427.682480,55.840632,-4.171166,55.837175,-4.170209,12,6,6,0.0


Provide a visualization that you used as part of your EDA process. Explain the initial pattern or relationship you discoved through this visualization. 

In [4]:
from folium.plugins import HeatMap
import folium

In [24]:
# place density and station locations
glasgow_coords = (55.8617,-4.2583)
glasgow_map = folium.Map(location=glasgow_coords, zoom_start=12.5)

heatmap_data = places_df[['place_latitude','place_longitude']]
heatmap_data = [[row['place_latitude'],row['place_longitude']] for index, row in heatmap_data.iterrows()]
for row in bikes_df.iterrows():
    row_value = row[1]
    location = [row_value['station_latitude'],row_value['station_longitude']]
    popup = f'<strong> Name: {row_value['station_name']}</strong><br> Number : {row_value['station_number']}'
    marker = folium.Marker(location =location,popup=popup)
    marker.add_to(glasgow_map)

HeatMap(heatmap_data).add_to(glasgow_map)
glasgow_map.save(r'images/glasgow_map.png')
glasgow_map

# Database

Put all your results in an SQLite3 database (remember, SQLite stores its databases as files in your local machine - make sure to create your database in your project's data/ directory!)

In [6]:
import sqlite3
from sqlite3 import Error

In [7]:
def create_connection(path):
    connection = None
    try: 
        connection = sqlite3.connect(path)
        print("Connection to SQLite DB successful")
    except Error as e:
        print(f"The error '{e}' occurred")

    return connection

def execute_query(connection, query):
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        connection.commit()
        print("Query executed successfully")
    except Error as e:
        print(f"The error '{e}' occurred")

def execute_insert_cvs(connection, query,contents):
    cursor = connection.cursor()
    try:
        cursor.execute(query,contents)
        connection.commit()
        print("Query executed successfully")
    except Error as e:
        print(f"The error '{e}' occurred")


In [8]:
connection = create_connection(r'B:\LHL\StatsModeling_Project\LHL-statsModeling-priject\data\Database\glasgowBikes.db')

Connection to SQLite DB successful


In [9]:
create_table_bikes = '''
                CREATE TABLE bike_stations(
                id TEXT PRIMARY KEY,
                name TEXT NOT NULL,
                latitude REAL NOT NULL,
                longitude REAL NOT NULL,
                free_bikes NUMERIC NOT NULL,
                empty_slots NUMERIC NOT NULL,
                station_number NUMERIC NOT NULL,
                slots NUMERIC NOT NULL);
                '''
create_table_places = '''
                CREATE TABLE places(
                id TEXT PRIMARY KEY,
                name TEXT NOT NULL,
                bar NUMERIC NOT NULL,
                dog_park NUMERIC NOT NULL,
                restaurant NUMERIC NOT NULL,
                distance REAL NOT NULL,
                latitude REAL NOT NULL,
                longitude REAL NOT NULL,
                rating REAL NOT NULL,
                station_id TEXT NOT NULL);
                '''
execute_query(connection,"DROP TABLE IF EXISTS bike_stations;")
execute_query(connection,create_table_bikes)
execute_query(connection,"DROP TABLE IF EXISTS places;")
execute_query(connection,create_table_places)

bikes_df.to_sql('bike_stations', connection, if_exists='replace',index=False)
places_df.to_sql('places', connection, if_exists='replace',index=False)

place='''SELECT * FROM places '''
sql_place = pd.read_sql_query(place, connection)
bikes='''SELECT * FROM bike_stations '''
sql_bikes = pd.read_sql_query(bikes, connection)



Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully


Look at the data before and after the join to validate your data.

In [10]:
sql_place

,place_id,place_name,bar,dog_park,restaurant,distance,place_latitude,place_longitude,place_rating,station_id
0,4b7b007cf964a5205b4b2fe3,Mother India's Cafe,0,0,1,931.000000,55.867287,-4.291100,NaN,066c99293af108ece27d9b0436c30cc4
1,54c61ff3498ec3e20f6a64b2,Meadow Road Coffee,0,0,1,902.000000,55.870559,-4.316758,NaN,066c99293af108ece27d9b0436c30cc4
2,56752941498ef438347e5c5e,Tantrum Doughnuts,0,0,1,851.000000,55.866930,-4.291825,NaN,066c99293af108ece27d9b0436c30cc4
3,57f54b2d498e574bd31426ea,Baffo,0,0,1,908.000000,55.867785,-4.291606,NaN,066c99293af108ece27d9b0436c30cc4
4,5635275a498e569169b27825,Kimchi Cult,0,0,1,800.000000,55.871878,-4.298324,NaN,066c99293af108ece27d9b0436c30cc4
...,...,...,...,...,...,...,...,...,...,...
1180,LOEGJ9C8QgEfbVq-haTwqA,Lincoln Inn,1,0,0,553.814985,55.899938,-4.349245,2.0,f6210447e1724487afe0ce63b011aecd
1181,B2B1zS_JHzOQvm9bJ9MQGA,New Dragon Inn,1,0,0,818.569003,55.896633,-4.341972,0.0,f6210447e1724487afe0ce63b011aecd
1182,M7ekLvEjNuqrx828SFFhCw,Kimberley,1,0,0,678.130598,55.843273,-4.170313,0.0,f6b64fd82fe6e607f1fc788ace31c3a1
1183,-vLT5nRX8i3miXL6Hhq6jw,Elders Bar,1,0,0,427.682480,55.840632,-4.171166,0.0,f6b64fd82fe6e607f1fc788ace31c3a1


In [11]:
sql_bikes

,station_id,station_name,station_latitude,station_longitude,free_bikes,empty_slots,station_number,slots
0,066c99293af108ece27d9b0436c30cc4,Riverside Museum,55.865683,-4.305367,6,0,8416,6
1,0a106cbc148d5a0c2535b51c1dbe3b4d,Parkhead,55.846524,-4.197475,4,4,8470,8
2,0cc8c09950e1435ee7782478ed292fef,Tollcross International Swimming Centre,55.844250,-4.176167,5,1,8200,6
3,0e94d2ad012bff0cf23497963ff5fd77,Shields Road,55.845807,-4.275232,5,3,8403,8
4,101c6cd7749f373507e9444f5222b7f2,Shawlands Shopping Centre (Kilmarnock Road),55.829057,-4.282675,4,3,8453,8
...,...,...,...,...,...,...,...,...
108,f6b64fd82fe6e607f1fc788ace31c3a1,Fullarton,55.837175,-4.170209,6,6,8234,12
109,f806566ccd2263bd5dbb624cdd0fb7f3,Bridgeton,55.848939,-4.226823,7,0,8441,8
110,f93ab2c803d81c14f7e2f1765911de04,Charing Cross Railway Station,55.864563,-4.270055,1,7,8421,8
111,fa6bc682e8627d66891c2b6250c81b1e,Mitchell Library,55.864788,-4.271852,0,6,8425,6


In [12]:
join = '''SELECT * FROM bike_stations
JOIN places ON bike_stations.station_id=places.station_id'''
sql_all = pd.read_sql_query(join, connection)
sql_all

,station_id,station_name,station_latitude,station_longitude,free_bikes,empty_slots,station_number,slots,place_id,place_name,bar,dog_park,restaurant,distance,place_latitude,place_longitude,place_rating,station_id
0,066c99293af108ece27d9b0436c30cc4,Riverside Museum,55.865683,-4.305367,6,0,8416,6,0f8bda6b6c6448584631c039,Partick Tavern,1,0,0,626.0,55.870642,-4.300599,NaN,066c99293af108ece27d9b0436c30cc4
1,066c99293af108ece27d9b0436c30cc4,Riverside Museum,55.865683,-4.305367,6,0,8416,6,0f9eb7a50a2c48fc1462d423,Bibimbap West End,0,0,1,587.0,55.869998,-4.299878,NaN,066c99293af108ece27d9b0436c30cc4
2,066c99293af108ece27d9b0436c30cc4,Riverside Museum,55.865683,-4.305367,6,0,8416,6,4b71383ef964a520023d2de3,Banana Leaf,0,0,1,804.0,55.867306,-4.292607,NaN,066c99293af108ece27d9b0436c30cc4
3,066c99293af108ece27d9b0436c30cc4,Riverside Museum,55.865683,-4.305367,6,0,8416,6,4b7b007cf964a5205b4b2fe3,Mother India's Cafe,0,0,1,931.0,55.867287,-4.291100,NaN,066c99293af108ece27d9b0436c30cc4
4,066c99293af108ece27d9b0436c30cc4,Riverside Museum,55.865683,-4.305367,6,0,8416,6,4b8135e7f964a520949a30e3,McDonald's,0,0,1,875.0,55.872141,-4.313057,NaN,066c99293af108ece27d9b0436c30cc4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1180,f806566ccd2263bd5dbb624cdd0fb7f3,Bridgeton,55.848939,-4.226823,7,0,8441,8,4ecff769e5faa5ec0246f890,Cabin Bar,1,0,0,994.0,55.855199,-4.236101,NaN,f806566ccd2263bd5dbb624cdd0fb7f3
1181,f806566ccd2263bd5dbb624cdd0fb7f3,Bridgeton,55.848939,-4.226823,7,0,8441,8,53cc2fb5498e10c40b24046b,The Traders Tavern,1,0,0,924.0,55.854630,-4.237765,NaN,f806566ccd2263bd5dbb624cdd0fb7f3
1182,f806566ccd2263bd5dbb624cdd0fb7f3,Bridgeton,55.848939,-4.226823,7,0,8441,8,53f3bb6f498e257ee8cb8615,Dannys Fast Food,0,0,1,921.0,55.855545,-4.236496,NaN,f806566ccd2263bd5dbb624cdd0fb7f3
1183,f806566ccd2263bd5dbb624cdd0fb7f3,Bridgeton,55.848939,-4.226823,7,0,8441,8,58fb5afa356b4972b023ab90,A'challatainn,0,0,1,939.0,55.855151,-4.236940,NaN,f806566ccd2263bd5dbb624cdd0fb7f3
